In [1]:
import numpy as np
import pandas as pd
import json
from pandas.io.json import json_normalize 
from pandas import read_csv
import logging
from functools import reduce
import csv

/Users/ghost/src/virtual_env/jupyther3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/ghost/src/virtual_env/jupyther3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## Configuration 

In [2]:
#etl = 'kobo2elastic'
etl = 'curis2elastic'
#etl = 'oldcuris2newcuris'

input_schema_file = ''
input_data_file = ''
mapping_file = ''
    
if etl == 'curis2elastic':
    #old curis to elasticsearch
    input_schema_file = 'schema/input/curisSchema.1-item.json'
    input_data_file = 'data/curisData.2-actual-items.json'
    mapping_file = 'schema/map/couchbase2elastic.map.csv'
elif etl == 'kobo2elastic':
    #kobo to elasticsearch
    input_schema_file = 'schema/input/aqmSchema.complete.json'
    input_data_file = 'data/aqmData.2-items.json'
    mapping_file = 'schema/map/kobo2elastic.map.csv'
elif etl == 'oldcuris2newcuris':
    #kobo to elasticsearch
    input_schema_file = 'schema/input/curisData.1-Schema.avro.json'
    input_data_file = 'data/curisData.1-items.json'
    mapping_file = 'schema/map/kobo2elastic.map.csv'

## Flatten json

In [3]:

def flatten_json(nested_json):
    """
        Flatten json object with nested keys into a single level.
        Args:
            nested_json: A nested json object.
        Returns:
            The flattened json object if successful, None otherwise.
    """
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '.')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + '' + str(i) + '.')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(nested_json)
    return out

## Load data

In [4]:
dx = []
dx = {}
dx_list = []
dx_dict = {}
dx_normalize_df = pd.DataFrame()

with open(input_data_file) as f:
    dx_list = json.load(f)

## Flatten input data

## TODO: Iterate here if multiple arrays

In [5]:
dx_flatten = flatten_json(dx_list[0])
dx_flatten

{'awh_id': '000210ae-ddfd-42f2-b79b-01908c4b73ef',
 'address.barangay': 'naganacan',
 'address.country': 'Philippines',
 'address.lot_or_house_number': '234234',
 'address.postal_code': '9230',
 'address.province': 'santa maria isabela',
 'birthdate': '09/19/1951',
 'contact_number.country_code': '+63',
 'contact_number.number': '23423432',
 'date_visits.0': '09/19/2018 at 03:44:06 PM GMT+08:00',
 'date_visits.1': '10/22/2018 at 09:44:17 AM GMT+08:00',
 'date_visits.2': '11/27/2018 at 06:21:14 PM GMT+08:00',
 'date_visits.3': '11/29/2018 at 07:30:11 PM GMT+08:00',
 'date_visits.4': '12/04/2018 at 09:41:21 AM GMT+08:00',
 'date_visits.5': '02/14/2019 at 02:25:29 PM GMT+08:00',
 'email_address': 'yahoo@gmail.com',
 'family_members.0': '5289d20e-c80f-4c9e-9e79-7cd3cc2a3e90',
 'family_members.1': '000210ae-ddfd-42f2-b79b-01908c4b73ef',
 'family_members.2': '6a0c8e9c-dfc6-48c8-bf1e-19a7f15229ec',
 'first_name': 'Rosita',
 'gender': 'Female',
 'health_informations.0.allergies': 'None',
 'hea

## Create index_map and key headers

In [6]:
## OPTIMIZE TO ACCEPT LIST NOT DATAFRAME
def filter_key(x):
    lists = format_key(x)
    lists = exclude_digit(lists)
    lists = list2string(lists)
    return lists

def filter_index_map(x):
    lists = format_key(x)
    lists = include_digit(lists)
    lists = list2string(lists)
    return lists

def format_key(items):
    #return list(map(lambda x:x.lower().split(sep='.'), items ))
    return items.lower().split(sep='.')

def include_digit(items):
    return [item for item in items if item.isdigit()]

def exclude_digit(items):
    return [item for item in items if not item.isdigit()]

def list2string(lists):
    return '.'.join(lists)

## Declare input data as Dataframe

In [7]:
dx_flatten_df = pd.DataFrame.from_dict({'value': dx_flatten})
dx_flatten_df

,value
address.barangay,naganacan
address.country,Philippines
address.lot_or_house_number,234234
address.postal_code,9230
address.province,santa maria isabela
awh_id,000210ae-ddfd-42f2-b79b-01908c4b73ef
birthdate,09/19/1951
contact_number.country_code,+63
contact_number.number,23423432
date_visits.0,09/19/2018 at 03:44:06 PM GMT+08:00


## Add _index_map and _id

In [8]:
dx_flatten_df['key'] = list(dx_flatten_df.index)
dx_flatten_df['key'] = dx_flatten_df['key'].apply(filter_key)

dx_flatten_df['_index_map'] = list(dx_flatten_df.index)
dx_flatten_df['_index_map'] = dx_flatten_df['_index_map'].apply(filter_index_map)

dx_flatten_df['_id'] = dx_flatten_df[dx_flatten_df['key'] == 'awh_id']['value'].values[0]

dx_flatten_df = dx_flatten_df.reset_index(drop=True)

dx_flatten_df

,value,key,_index_map,_id
0,naganacan,address.barangay,,000210ae-ddfd-42f2-b79b-01908c4b73ef
1,Philippines,address.country,,000210ae-ddfd-42f2-b79b-01908c4b73ef
2,234234,address.lot_or_house_number,,000210ae-ddfd-42f2-b79b-01908c4b73ef
3,9230,address.postal_code,,000210ae-ddfd-42f2-b79b-01908c4b73ef
4,santa maria isabela,address.province,,000210ae-ddfd-42f2-b79b-01908c4b73ef
5,000210ae-ddfd-42f2-b79b-01908c4b73ef,awh_id,,000210ae-ddfd-42f2-b79b-01908c4b73ef
6,09/19/1951,birthdate,,000210ae-ddfd-42f2-b79b-01908c4b73ef
7,+63,contact_number.country_code,,000210ae-ddfd-42f2-b79b-01908c4b73ef
8,23423432,contact_number.number,,000210ae-ddfd-42f2-b79b-01908c4b73ef
9,09/19/2018 at 03:44:06 PM GMT+08:00,date_visits,0,000210ae-ddfd-42f2-b79b-01908c4b73ef


## Filter data for main @root

In [9]:
root_data_df = pd.DataFrame()
root_data_df = dx_flatten_df[dx_flatten_df['_index_map'] == '' ]
root_data_df

,value,key,_index_map,_id
0,naganacan,address.barangay,,000210ae-ddfd-42f2-b79b-01908c4b73ef
1,Philippines,address.country,,000210ae-ddfd-42f2-b79b-01908c4b73ef
2,234234,address.lot_or_house_number,,000210ae-ddfd-42f2-b79b-01908c4b73ef
3,9230,address.postal_code,,000210ae-ddfd-42f2-b79b-01908c4b73ef
4,santa maria isabela,address.province,,000210ae-ddfd-42f2-b79b-01908c4b73ef
5,000210ae-ddfd-42f2-b79b-01908c4b73ef,awh_id,,000210ae-ddfd-42f2-b79b-01908c4b73ef
6,09/19/1951,birthdate,,000210ae-ddfd-42f2-b79b-01908c4b73ef
7,+63,contact_number.country_code,,000210ae-ddfd-42f2-b79b-01908c4b73ef
8,23423432,contact_number.number,,000210ae-ddfd-42f2-b79b-01908c4b73ef
15,yahoo@gmail.com,email_address,,000210ae-ddfd-42f2-b79b-01908c4b73ef


## Get CSV headers for main @root

In [10]:
root_csv_df = pd.DataFrame()
root_csv_df = pd.read_csv('file/resident.csv',nrows=0) #get header only
root_csv_df
#object_array_csv_df.to_csv('file/' + header + '.csv', encoding='utf-8', mode='a', header=False,index=False)

,awh_id,birthdate,date_visits,email_address,family_members,first_name,gender,last_name,last_name_suffix,middle_name,...,contact_number.country_code,profile_picture.name,contact_number.number,user-cam.id,profile_picture.path,user-cam.owner,identification.id2.identifier,identification.id2.type,_id,_index_map


## Get input data with headers for main @root

In [11]:
required_header_list = [] 

for data in root_data_df['key']:
    if data in root_csv_df.columns:
        required_header_list.append(data)
required_header_list

['address.barangay',
 'address.country',
 'address.lot_or_house_number',
 'address.postal_code',
 'address.province',
 'awh_id',
 'birthdate',
 'contact_number.country_code',
 'contact_number.number',
 'email_address',
 'first_name',
 'gender',
 'identification.id1.identifier',
 'identification.id1.type',
 'identification.id2.identifier',
 'identification.id2.type',
 'identification.id3.identifier',
 'identification.id3.type',
 'last_name',
 'last_name_suffix',
 'middle_name',
 'nhid',
 'organization',
 'profile_picture.name',
 'profile_picture.path',
 'registered_at',
 'type',
 'user-cam.id',
 'user-cam.owner']

In [12]:
new_data_df = pd.DataFrame()
new_data_df = root_data_df[root_data_df['key'].isin(required_header_list)]

new_data_df

,value,key,_index_map,_id
0,naganacan,address.barangay,,000210ae-ddfd-42f2-b79b-01908c4b73ef
1,Philippines,address.country,,000210ae-ddfd-42f2-b79b-01908c4b73ef
2,234234,address.lot_or_house_number,,000210ae-ddfd-42f2-b79b-01908c4b73ef
3,9230,address.postal_code,,000210ae-ddfd-42f2-b79b-01908c4b73ef
4,santa maria isabela,address.province,,000210ae-ddfd-42f2-b79b-01908c4b73ef
5,000210ae-ddfd-42f2-b79b-01908c4b73ef,awh_id,,000210ae-ddfd-42f2-b79b-01908c4b73ef
6,09/19/1951,birthdate,,000210ae-ddfd-42f2-b79b-01908c4b73ef
7,+63,contact_number.country_code,,000210ae-ddfd-42f2-b79b-01908c4b73ef
8,23423432,contact_number.number,,000210ae-ddfd-42f2-b79b-01908c4b73ef
15,yahoo@gmail.com,email_address,,000210ae-ddfd-42f2-b79b-01908c4b73ef


## Drop key column name

In [13]:
if 'key' in new_data_df.columns:
    new_data_df.index = list(new_data_df['key'])
    new_data_df = new_data_df.drop('key', axis=1)
new_data_df 

,value,_index_map,_id
address.barangay,naganacan,,000210ae-ddfd-42f2-b79b-01908c4b73ef
address.country,Philippines,,000210ae-ddfd-42f2-b79b-01908c4b73ef
address.lot_or_house_number,234234,,000210ae-ddfd-42f2-b79b-01908c4b73ef
address.postal_code,9230,,000210ae-ddfd-42f2-b79b-01908c4b73ef
address.province,santa maria isabela,,000210ae-ddfd-42f2-b79b-01908c4b73ef
awh_id,000210ae-ddfd-42f2-b79b-01908c4b73ef,,000210ae-ddfd-42f2-b79b-01908c4b73ef
birthdate,09/19/1951,,000210ae-ddfd-42f2-b79b-01908c4b73ef
contact_number.country_code,+63,,000210ae-ddfd-42f2-b79b-01908c4b73ef
contact_number.number,23423432,,000210ae-ddfd-42f2-b79b-01908c4b73ef
email_address,yahoo@gmail.com,,000210ae-ddfd-42f2-b79b-01908c4b73ef


In [14]:
for header in list(new_data_df.index):
    root_csv_df.at['',header] = new_data_df.loc[header]['value'] 

root_csv_df.at['','_id'] = new_data_df['_id'][header] 
root_csv_df.at['','_index_map'] = new_data_df['_index_map'][header] 

## Write input data for main @root

In [15]:
root_csv_df.to_csv('file/' +'resident' + '.csv', encoding='utf-8', mode='a', header=False,index=False)
root_csv_df

,awh_id,birthdate,date_visits,email_address,family_members,first_name,gender,last_name,last_name_suffix,middle_name,...,contact_number.country_code,profile_picture.name,contact_number.number,user-cam.id,profile_picture.path,user-cam.owner,identification.id2.identifier,identification.id2.type,_id,_index_map
,000210ae-ddfd-42f2-b79b-01908c4b73ef,09/19/1951,NaN,yahoo@gmail.com,NaN,Rosita,Female,Antalan,Sr,Laforteza,...,+63,8c9ef45a-c0b0-4f0a-a1d8-8c6c11e6b895,23423432,lcorpuz@gmail.com,/data/data/com.awh.health.curis/app_images/ima...,lcorpuz@gmail.com,SSS,4Ps,000210ae-ddfd-42f2-b79b-01908c4b73ef,


## ===========================

## Get filenames

## Get index_map unique values

In [16]:
index_map_list = [] 
index_map_list = list(dx_flatten_df['_index_map'].unique())

#excluse root
index_map_list[1:]

['0', '1', '2', '3', '4', '5', '0.0', '0.1', '0.2']

## Iterate folder and get filenames 

In [17]:
schema_csv = pd.read_csv('file/schema.csv', skiprows=0)
filenames_list = list(schema_csv['file_name'])
filenames_list

['date_visits,family_members,health_informations.diagnosed,health_informations.family_history,households.amenities_present_in_house,households.sanitary_type,profiles,health_informations,households,resident']

## Filter data for non-root

In [18]:
non_root_data_df = pd.DataFrame()
non_root_data_df = dx_flatten_df[dx_flatten_df['_index_map'] == '2' ]
non_root_data_df

,value,key,_index_map,_id
11,11/27/2018 at 06:21:14 PM GMT+08:00,date_visits,2,000210ae-ddfd-42f2-b79b-01908c4b73ef
18,6a0c8e9c-dfc6-48c8-bf1e-19a7f15229ec,family_members,2,000210ae-ddfd-42f2-b79b-01908c4b73ef


## Get CSV headers for non-root

In [19]:
array_csv_df = pd.DataFrame()
array_csv_df = pd.read_csv('file/family_members.csv',nrows=0) #get header only
array_csv_df
#object_array_csv_df.to_csv('file/' + header + '.csv', encoding='utf-8', mode='a', header=False,index=False)

,family_members,_id,_index_map


## Get input data with headers for non-root

In [20]:
required_header_list = [] 

for data in non_root_data_df['key']:
    if data in array_csv_df.columns:
        required_header_list.append(data)
required_header_list

['family_members']

## Write input data for main non-root

In [21]:
new_array_data_df = pd.DataFrame()
new_array_data_df = non_root_data_df[non_root_data_df['key'].isin(required_header_list)]

new_array_data_df

,value,key,_index_map,_id
18,6a0c8e9c-dfc6-48c8-bf1e-19a7f15229ec,family_members,2,000210ae-ddfd-42f2-b79b-01908c4b73ef


In [22]:
if 'key' in new_array_data_df.columns:
    new_array_data_df.index = list(new_array_data_df['key'])
    new_array_data_df = new_array_data_df.drop('key', axis=1)

new_array_data_df

,value,_index_map,_id
family_members,6a0c8e9c-dfc6-48c8-bf1e-19a7f15229ec,2,000210ae-ddfd-42f2-b79b-01908c4b73ef


In [23]:
for header in list(new_array_data_df.index):
    array_csv_df.at['',header] = new_array_data_df.loc[header]['value'] 

array_csv_df.at['','_id'] = new_array_data_df['_id'][header] 
array_csv_df.at['','_index_map'] = new_array_data_df['_index_map'][header] 
array_csv_df

,family_members,_id,_index_map
,6a0c8e9c-dfc6-48c8-bf1e-19a7f15229ec,000210ae-ddfd-42f2-b79b-01908c4b73ef,2


In [24]:
array_csv_df.to_csv('file/family_members' + '.csv', encoding='utf-8', mode='a', header=False,index=False)
array_csv_df

,family_members,_id,_index_map
,6a0c8e9c-dfc6-48c8-bf1e-19a7f15229ec,000210ae-ddfd-42f2-b79b-01908c4b73ef,2
